In [1]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

from sapaicore_utils import get_conn_details, onboard_repository, show_repositories, GITHUB_TOKEN, GITHUB_USER, DOCKER_TOKEN, DOCKER_USER

In [2]:
conn_details = get_conn_details()
ai_core_client = AICoreV2Client(**conn_details)

## Step 0. Onboard a repository and Docker account

### GitHub repository
We'll need a GitHub username and token

In [ ]:
PROJECT_NAME = 'gsalomone-tf-gpu'
onboard_repository('https://github.com/gianfranco-s/SAP-aicore-gpu',
                    GITHUB_USER,
                    GITHUB_TOKEN,
                    PROJECT_NAME,
                    ai_core_client)

In [3]:
show_repositories(ai_core_client)

Name: sap-training-ncellini
URL: https://github.com/NCellini/aicore-pipelines
Status: RepositoryStatus.COMPLETED

Name: gsalomone-tf-gpu
URL: https://github.com/gianfranco-s/SAP-aicore-gpu
Status: RepositoryStatus.COMPLETED



### Docker account


In [ ]:
res.__dict__

## Step 1. Create workspace
Using default resource group because when I tried to create `tf-demo`, I got the message "Resource Group cannot be created for free tier tenant"

In [ ]:
try:
    response = ai_core_client.resource_groups.create("tf-demo")

except Exception as e:
    print(e)


In [ ]:
def show_resource_groups() -> None:
    response = ai_core_client.resource_groups.query()

    for rg in response.resources:
        print(rg.resource_group_id)

show_resource_groups()

## Step 2: upload model files to AWS S3
Create bucket
```
aws s3api create-bucket --bucket gsalomone-celestial-bucket --region us-east-1
```

Upload files
```
cd tf_files
aws s3 cp model.h5 s3://gsalomone-celestial-bucket/movie-clf/model/  
aws s3 cp max_pad_len.txt s3://gsalomone-celestial-bucket/movie-clf/model/  
aws s3 cp label_encoded_classes.npy s3://gsalomone-celestial-bucket/movie-clf/model/  
aws s3 cp tokens.json s3://gsalomone-celestial-bucket/movie-clf/model/  
```

Check files
```
aws s3 ls s3://gsalomone-celestial-bucket/movie-clf/model/
```

## Step 3: connect AWS S3 to SAP AI Core


In [ ]:
aws_credentials_file = '/home/gsalomone/Documents/gian-dev_accessKeys.csv'

def get_aws_credentials(filepath: str) -> tuple:
    with open(filepath, 'r') as f:
        creds = f.readlines()[1:2][0]
        aws_access_key_id, aws_secret_access_key = creds.split(',')
        return aws_access_key_id, aws_secret_access_key

aws_access_key_id, aws_secret_access_key = get_aws_credentials(aws_credentials_file)
aws_az = 'us-east-1'
object_store_name = 'gs-tf-gpu-tutorial-secret'

In [ ]:

def create_s3_object_store() -> str:
    response = ai_core_client.object_store_secrets.create(
        resource_group = 'default',
        type = "S3",
        name = object_store_name,
        path_prefix = "movie-clf",
        endpoint = f"s3-{aws_az}.amazonaws.com",
        bucket = "gsalomone-celestial-bucket",
        region = aws_az,
        data = {
            "AWS_ACCESS_KEY_ID": aws_access_key_id,
            "AWS_SECRET_ACCESS_KEY": aws_secret_access_key
        }
    )

    return response.message

res = create_s3_object_store()
res

In [ ]:
github_token_file = 'github.token'

def get_github_token(filepath: str) -> str:
    with open(filepath, 'r') as f:
        return f.readline().split('=')[1]

github_token = get_github_token(github_token_file)

In [ ]:
# Onboard a repository
github_repo_url = 'https://github.com/gianfranco-s/SAP-aicore-gpu'
github_account_usr = 'gsalomone'
project_name = 'sap-gpu-gsalomone'

res = ai_core_client.repositories.create(
        name=project_name,
        url=github_repo_url,
        username=github_account_usr,
        password=github_token
)

print(res)

In [ ]:
# Create application
application_name = 'gsalomone-tf-app'
response = ai_core_client.applications.create(
    application_name = application_name,
    revision = "HEAD",
    repository_url = github_repo_url,
    path = "tutorial-tf-text-clf"
)

print(response.__dict__)

In [ ]:
response = ai_core_client.applications.get_status(application_name=application_name)

print(response.message)
for workflow_sync_status in response.sync_ressources_status:
    print(workflow_sync_status.__dict__)

In [ ]:
# Register model as artifact
from ai_core_sdk.models import Artifact

response = ai_core_client.artifact.create(
    resource_group = 'default',
    name = "tf-txt-clf-model",
    kind = Artifact.Kind.MODEL,
    url = f"ai://{object_store_name}/model",
    scenario_id = "tf-text-clf",  # same as under `scenarios.ai.sap.com/id`
    description = "Review Classification Model"
)

print(response.__dict__)

In [ ]:
artifact_id = response.__dict__.get('id')
artifact_id

In [ ]:
# Start deployment

# Create configuration
from ai_core_sdk.models import InputArtifactBinding

response = ai_core_client.configuration.create(
    name = "gsalomone-tf-gpu-conf",
    resource_group = "default",
    scenario_id = "tf-text-clf",
    executable_id = "tf-text-clf-serve",
    input_artifact_bindings = [
        # list
        InputArtifactBinding(key="modelArtifact", artifact_id = artifact_id), # Change artifact id
    ]
)

print(response.__dict__)
config_id = response.__dict__.get('id')

In [ ]:
# Create deployment
response = ai_core_client.deployment.create(
    resource_group = "default",
    configuration_id = config_id # change this
)
print(response.__dict__)
deployment_id = response.__dict__.get('id')
deployment_id

In [ ]:
# Check deployment status
response = ai_core_client.deployment.get(
    resource_group = "default",
    deployment_id = deployment_id # Change this
)

print("Status: ", response.status)
print('*'*80)
pprint(response.__dict__)

In [ ]:
# SECOND TIME: check deployment status
response = ai_core_client.deployment.get(
    resource_group = "default",
    deployment_id = deployment_id # Change this
)

print("Status: ", response.status)
print('*'*80)
from pprint import pprint
pprint(response.__dict__)